# Part III — Real Projects  
## 17. Project 2 (Intermediate): Multi‑User App (Team Tasks / Mini‑CRM Style)

This project teaches the patterns you *must* know to build real internal tools and
multi-user business apps in Django:

- **scoping data** (users should only see their organization’s records)
- **roles + permissions** (admins vs members)
- **object-level authorization** (owner/assignee rules)
- **audit fields** (`created_by`, `updated_by`, timestamps)
- **filter/search UI** using GET query params + pagination
- **CSV export** for reporting (industry standard)
- **tests** that prove permissions and scoping

We’ll build a “Team Tasks” app with organizations and membership.

You’ll end with URLs like:

- `/orgs/acme/tasks/` (list tasks in org “acme”)
- `/orgs/acme/tasks/new/` (create task)
- `/orgs/acme/tasks/123/` (view task)
- `/orgs/acme/tasks/123/edit/` (edit task)
- `/orgs/acme/tasks/export.csv?status=open&assigned_to=me` (export)

---

## 17.0 What you’ll build (Features)

### Organization + membership
- Users can belong to multiple orgs
- Each membership has a role:
  - **ADMIN**: can manage org data and members (in admin)
  - **MEMBER**: can create tasks and manage their own/assigned tasks

### Tasks
- A task belongs to exactly one org
- Fields: title, description, status, priority, due_date, assignee
- Audit: created_by / updated_by / created_at / updated_at
- Permissions:
  - Any org member can view org tasks
  - Only ADMINs can edit any task
  - MEMBERs can edit tasks they created **or** tasks assigned to them
  - Only ADMINs can delete (simple rule)

### UX
- Task list includes:
  - search (`q`)
  - filter by status/priority/assignee
  - “me” filter for assignee
  - pagination that preserves filters

### Export
- CSV export with same filters as list
- Permission required (ADMIN only, or “export” permission)

---

## 17.1 Requirements & User Stories (Write these; they drive your design)

### Org/membership
1. As a user, I can belong to an org.
2. As an admin, I can add members to an org (we’ll do this in Django admin first).

### Tasks
3. As an org member, I can list tasks in my org.
4. As an org member, I can create a task in my org.
5. As an org member, I can view a task in my org.
6. As an admin, I can edit/delete any task in my org.
7. As a member, I can edit tasks I created or tasks assigned to me.
8. As a non-member, I cannot access an org’s task URLs (404 or 403—choose policy).

### Reporting
9. As an admin, I can export tasks to CSV with filters.

### Quality
10. As a developer, I can prove scoping and permissions via automated tests.

---

## 17.2 Data Model Design (Industry‑Standard Scoping)

### Why an Organization model is important
If you skip org scoping and just add `owner=user` everywhere, you can’t represent:
- teams
- shared work
- role-based rules
- data boundaries for companies (multi-tenant SaaS)

So we’ll do:

- `Organization`
- `Membership` (org ↔ user with role)
- `Task` (belongs to org, has assignee, audit fields)

---

## 17.3 Create Apps: `orgs` and `tasks`

Run:

```bash
python manage.py startapp orgs
python manage.py startapp tasks
```

Add them to `INSTALLED_APPS` in `config/settings.py`:

```python
INSTALLED_APPS = [
    # ...
    "orgs",
    "tasks",
]
```

---

## 17.4 Build Models (With Constraints + Indexes)

### 17.4.1 `orgs/models.py`

```python
from django.conf import settings
from django.db import models


class Organization(models.Model):
    name = models.CharField(max_length=120)
    slug = models.SlugField(max_length=140, unique=True)

    created_at = models.DateTimeField(auto_now_add=True)

    class Meta:
        ordering = ["name"]

    def __str__(self) -> str:
        return self.name


class Membership(models.Model):
    class Role(models.TextChoices):
        ADMIN = "admin", "Admin"
        MEMBER = "member", "Member"

    organization = models.ForeignKey(
        Organization,
        on_delete=models.CASCADE,
        related_name="memberships",
    )
    user = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.CASCADE,
        related_name="memberships",
    )
    role = models.CharField(
        max_length=20,
        choices=Role.choices,
        default=Role.MEMBER,
    )

    created_at = models.DateTimeField(auto_now_add=True)

    class Meta:
        constraints = [
            models.UniqueConstraint(
                fields=["organization", "user"],
                name="unique_org_user_membership",
            ),
        ]
        indexes = [
            models.Index(fields=["organization", "role"]),
            models.Index(fields=["user"]),
        ]

    def __str__(self) -> str:
        return f"{self.user} in {self.organization} ({self.role})"
```

#### Why these design choices matter

- `Organization.slug` unique:
  - stable URL key: `/orgs/acme/`
  - avoids leaking numeric IDs
- `Membership` is a join model instead of ManyToMany:
  - because we need extra fields (role, created_at)
- Unique constraint:
  - prevents duplicate memberships
  - protects integrity even under concurrent requests

---

### 17.4.2 `tasks/models.py` (Task + audit fields)

```python
from django.conf import settings
from django.db import models

from orgs.models import Organization


class Task(models.Model):
    class Status(models.TextChoices):
        OPEN = "open", "Open"
        IN_PROGRESS = "in_progress", "In progress"
        DONE = "done", "Done"
        CANCELED = "canceled", "Canceled"

    class Priority(models.IntegerChoices):
        LOW = 1, "Low"
        MEDIUM = 2, "Medium"
        HIGH = 3, "High"
        URGENT = 4, "Urgent"

    organization = models.ForeignKey(
        Organization,
        on_delete=models.CASCADE,
        related_name="tasks",
    )

    title = models.CharField(max_length=200)
    description = models.TextField(blank=True)

    status = models.CharField(
        max_length=20,
        choices=Status.choices,
        default=Status.OPEN,
    )
    priority = models.IntegerField(
        choices=Priority.choices,
        default=Priority.MEDIUM,
    )

    due_date = models.DateField(null=True, blank=True)

    assigned_to = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.SET_NULL,
        null=True,
        blank=True,
        related_name="assigned_tasks",
    )

    created_by = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.PROTECT,
        related_name="created_tasks",
    )
    updated_by = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.PROTECT,
        related_name="updated_tasks",
    )

    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)

    class Meta:
        ordering = ["-created_at"]
        indexes = [
            models.Index(fields=["organization", "status", "-created_at"]),
            models.Index(fields=["organization", "assigned_to"]),
            models.Index(fields=["organization", "due_date"]),
        ]

    def __str__(self) -> str:
        return f"[{self.organization.slug}] {self.title}"
```

#### Why `created_by` and `updated_by` are PROTECT
If you delete a user, you typically don’t want to delete tasks or lose audit trails.
`PROTECT` makes deletion fail unless tasks are reassigned (explicit decision).

#### Why `assigned_to` is SET_NULL
If an assignee leaves, tasks should remain, but become unassigned rather than
deleted.

---

## 17.5 Migrations

Run:

```bash
python manage.py makemigrations
python manage.py migrate
```

---

## 17.6 Admin Setup (So You Can Manage Orgs & Memberships Quickly)

### 17.6.1 `orgs/admin.py`

```python
from django.contrib import admin

from .models import Membership, Organization


@admin.register(Organization)
class OrganizationAdmin(admin.ModelAdmin):
    list_display = ("id", "name", "slug", "created_at")
    search_fields = ("name", "slug")
    prepopulated_fields = {"slug": ("name",)}


@admin.register(Membership)
class MembershipAdmin(admin.ModelAdmin):
    list_display = ("id", "organization", "user", "role", "created_at")
    list_filter = ("role", "organization")
    search_fields = ("organization__name", "user__username", "user__email")
```

### 17.6.2 `tasks/admin.py`

```python
from django.contrib import admin

from .models import Task


@admin.register(Task)
class TaskAdmin(admin.ModelAdmin):
    list_display = (
        "id",
        "organization",
        "title",
        "status",
        "priority",
        "assigned_to",
        "due_date",
        "created_by",
        "updated_by",
        "created_at",
    )
    list_filter = ("organization", "status", "priority", "due_date")
    search_fields = ("title", "description")
    ordering = ("-created_at",)
```

---

## 17.7 Authorization Helpers (Centralize “Org Membership Required”)

Professional Django projects centralize access checks; they don’t copy/paste them in
every view.

Create `orgs/services.py`:

```python
from django.http import Http404

from .models import Membership, Organization


def get_org_for_user_or_404(*, user, org_slug: str) -> Organization:
    """
    Returns the org if user is a member, else raises 404.
    Using 404 is a common choice to avoid revealing org existence.
    """
    try:
        org = Organization.objects.get(slug=org_slug)
    except Organization.DoesNotExist as e:
        raise Http404("Organization not found") from e

    is_member = Membership.objects.filter(
        organization=org,
        user=user,
    ).exists()

    if not is_member:
        raise Http404("Organization not found")

    return org


def get_membership(*, user, organization: Organization) -> Membership:
    return Membership.objects.get(user=user, organization=organization)


def is_org_admin(*, membership: Membership) -> bool:
    return membership.role == Membership.Role.ADMIN
```

### Why 404 instead of 403?
Both are valid. Common choices:
- 404: “you don’t even know this org exists” (security-through-reduction)
- 403: “org exists but you’re forbidden” (more transparent)

We use 404 because it’s common in multi-tenant apps.

---

## 17.8 Forms (Task create/edit + list filters)

### 17.8.1 `tasks/forms.py` (ModelForm)

```python
from django import forms

from .models import Task


class TaskForm(forms.ModelForm):
    class Meta:
        model = Task
        fields = [
            "title",
            "description",
            "status",
            "priority",
            "due_date",
            "assigned_to",
        ]
        widgets = {
            "description": forms.Textarea(attrs={"rows": 6}),
            "due_date": forms.DateInput(attrs={"type": "date"}),
        }

    def clean_title(self) -> str:
        title = (self.cleaned_data.get("title") or "").strip()
        if len(title) < 3:
            raise forms.ValidationError("Title must be at least 3 characters.")
        return title
```

### 17.8.2 Filter form for GET query params (clean + validated)

Create `tasks/forms_filters.py`:

```python
from django import forms

from tasks.models import Task


class TaskFilterForm(forms.Form):
    q = forms.CharField(required=False, max_length=100)
    status = forms.ChoiceField(
        required=False,
        choices=[("", "Any status")] + list(Task.Status.choices),
    )
    priority = forms.ChoiceField(
        required=False,
        choices=[("", "Any priority")] + list(Task.Priority.choices),
    )
    assigned_to = forms.CharField(
        required=False,
        max_length=50,
        help_text="Use 'me' to filter tasks assigned to you.",
    )
    page = forms.IntegerField(required=False, min_value=1)

    def clean_q(self) -> str:
        return (self.cleaned_data.get("q") or "").strip()
```

#### Why filter forms matter
Without them, you’ll repeatedly:
- parse ints
- handle empty strings
- validate choices
…and you’ll do it inconsistently.

This form:
- normalizes inputs
- blocks invalid page numbers
- keeps list view logic readable

---

## 17.9 Views (Scoped CRUD + Filtering + Pagination + Export)

We’ll implement these views:

- list tasks in org
- task detail
- create task
- edit task (owner/assignee/admin rule)
- delete task (admin only)
- export CSV (admin only)

### 17.9.1 URL routing (namespaced, org-scoped)

Create `tasks/urls.py`:

```python
from django.urls import path

from . import views

app_name = "tasks"

urlpatterns = [
    path("", views.task_list, name="list"),
    path("new/", views.task_create, name="create"),
    path("export.csv", views.task_export_csv, name="export_csv"),
    path("<int:task_id>/", views.task_detail, name="detail"),
    path("<int:task_id>/edit/", views.task_edit, name="edit"),
    path("<int:task_id>/delete/", views.task_delete, name="delete"),
]
```

Now create an org-scoped include in `orgs/urls.py`:

```python
from django.urls import include, path

app_name = "orgs"

urlpatterns = [
    path("<slug:org_slug>/tasks/", include("tasks.urls")),
]
```

Include in `config/urls.py`:

```python
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path("admin/", admin.site.urls),
    path("accounts/", include("django.contrib.auth.urls")),
    path("orgs/", include("orgs.urls")),
    path("", include("pages.urls")),
    path("articles/", include("articles.urls")),
]
```

Now tasks are always scoped by org slug in URL — very common in industry.

---

## 17.10 Task list view (filters + pagination + scoping)

Create `tasks/views.py`:

```python
import csv

from django.contrib.auth.decorators import login_required
from django.core.paginator import EmptyPage, PageNotAnInteger, Paginator
from django.db.models import Q
from django.http import Http404, HttpResponse
from django.shortcuts import get_object_or_404, redirect, render
from django.views.decorators.http import require_GET, require_http_methods

from orgs.services import get_membership, get_org_for_user_or_404, is_org_admin
from orgs.models import Membership
from tasks.forms import TaskForm
from tasks.forms_filters import TaskFilterForm
from tasks.models import Task
```

### 17.10.1 `task_list` implementation

```python
@login_required
@require_GET
def task_list(request, org_slug: str):
    org = get_org_for_user_or_404(user=request.user, org_slug=org_slug)
    membership = get_membership(user=request.user, organization=org)

    form = TaskFilterForm(request.GET)
    form.is_valid()

    q = form.cleaned_data.get("q") if form.is_valid() else ""
    status = form.cleaned_data.get("status") if form.is_valid() else ""
    priority = form.cleaned_data.get("priority") if form.is_valid() else ""
    assigned_to = form.cleaned_data.get("assigned_to") if form.is_valid() else ""
    page_number = form.cleaned_data.get("page") if form.is_valid() else 1

    qs = (
        Task.objects.filter(organization=org)
        .select_related("assigned_to", "created_by", "updated_by")
        .order_by("-created_at")
    )

    if q:
        qs = qs.filter(Q(title__icontains=q) | Q(description__icontains=q))

    if status:
        qs = qs.filter(status=status)

    if priority:
        qs = qs.filter(priority=int(priority))

    if assigned_to:
        if assigned_to == "me":
            qs = qs.filter(assigned_to=request.user)
        else:
            qs = qs.filter(assigned_to__username__iexact=assigned_to)

    paginator = Paginator(qs, 20)

    try:
        page_obj = paginator.page(page_number or 1)
    except PageNotAnInteger:
        page_obj = paginator.page(1)
    except EmptyPage:
        page_obj = paginator.page(paginator.num_pages)

    can_export = is_org_admin(membership=membership)

    return render(
        request,
        "tasks/list.html",
        {
            "organization": org,
            "membership": membership,
            "form": form,
            "tasks": page_obj.object_list,
            "page_obj": page_obj,
            "paginator": paginator,
            "can_export": can_export,
        },
    )
```

#### Explanation: why list is written this way

- **Scoping first**: `Task.objects.filter(organization=org)`
  - This prevents data leaks across orgs.
  - Never rely on “UI hiding”; enforce scoping in queries.
- **`select_related`**: we display assigned/created/updated users in template
  - avoids N+1 query explosion
- **Filters are optional** and applied only if provided
- **Pagination**:
  - protects performance
  - yields stable UX
  - avoids massive pages and timeouts

---

## 17.11 Task detail view (scoped access)

```python
@login_required
@require_GET
def task_detail(request, org_slug: str, task_id: int):
    org = get_org_for_user_or_404(user=request.user, org_slug=org_slug)

    task = get_object_or_404(
        Task.objects.select_related(
            "assigned_to",
            "created_by",
            "updated_by",
        ),
        id=task_id,
        organization=org,
    )

    return render(
        request,
        "tasks/detail.html",
        {"organization": org, "task": task},
    )
```

#### Why org scoping is in the object lookup
This is a critical security pattern:

```python
get_object_or_404(Task, id=task_id, organization=org)
```

If you did:

```python
task = Task.objects.get(id=task_id)
```

a user could guess IDs and access tasks from other orgs.

---

## 17.12 Create task view (audit fields + secure org assignment)

```python
@login_required
@require_http_methods(["GET", "POST"])
def task_create(request, org_slug: str):
    org = get_org_for_user_or_404(user=request.user, org_slug=org_slug)

    if request.method == "POST":
        form = TaskForm(request.POST)
        if form.is_valid():
            task = form.save(commit=False)

            task.organization = org
            task.created_by = request.user
            task.updated_by = request.user

            task.save()
            form.save_m2m()

            return redirect("tasks:detail", org_slug=org.slug, task_id=task.id)
    else:
        form = TaskForm()

    return render(
        request,
        "tasks/form.html",
        {"organization": org, "form": form, "mode": "create"},
    )
```

### Why `organization` must be set in view (not form)
If you let users pick organization via a form field, they can create tasks in orgs
they don’t belong to. This is a common authorization bug.

Correct pattern:
- org comes from the URL + membership check
- server sets it (never trust client input)

---

## 17.13 Edit task view (object-level authorization rule)

Rule:
- Admin can edit any org task
- Member can edit if they are:
  - creator (`created_by`)
  - assignee (`assigned_to`)

```python
from django.core.exceptions import PermissionDenied


@login_required
@require_http_methods(["GET", "POST"])
def task_edit(request, org_slug: str, task_id: int):
    org = get_org_for_user_or_404(user=request.user, org_slug=org_slug)
    membership = get_membership(user=request.user, organization=org)

    task = get_object_or_404(Task, id=task_id, organization=org)

    is_admin = is_org_admin(membership=membership)
    is_creator = task.created_by_id == request.user.id
    is_assignee = task.assigned_to_id == request.user.id

    if not (is_admin or is_creator or is_assignee):
        raise PermissionDenied("You cannot edit this task.")

    if request.method == "POST":
        form = TaskForm(request.POST, instance=task)
        if form.is_valid():
            task = form.save(commit=False)
            task.updated_by = request.user
            task.save()
            form.save_m2m()
            return redirect("tasks:detail", org_slug=org.slug, task_id=task.id)
    else:
        form = TaskForm(instance=task)

    return render(
        request,
        "tasks/form.html",
        {
            "organization": org,
            "form": form,
            "mode": "edit",
            "task": task,
        },
    )
```

#### Why we raise 403 here (PermissionDenied) but use 404 for org membership
- Org membership: we used 404 to avoid leaking org existence
- Once inside org scope (member), if you can’t edit a specific object, 403 is more
  semantically correct.

Both policies are defensible; consistency matters.

---

## 17.14 Delete task view (admin only)

```python
@login_required
@require_http_methods(["GET", "POST"])
def task_delete(request, org_slug: str, task_id: int):
    org = get_org_for_user_or_404(user=request.user, org_slug=org_slug)
    membership = get_membership(user=request.user, organization=org)

    if not is_org_admin(membership=membership):
        raise PermissionDenied("Only org admins can delete tasks.")

    task = get_object_or_404(Task, id=task_id, organization=org)

    if request.method == "POST":
        task.delete()
        return redirect("tasks:list", org_slug=org.slug)

    return render(
        request,
        "tasks/confirm_delete.html",
        {"organization": org, "task": task},
    )
```

---

## 17.15 CSV Export (Admin-only, filter-aware)

Export endpoints are extremely common in internal tools. The main rules:
- apply the same scoping rules as the UI
- apply the same filters as the list view
- protect access (admin-only or explicit permission)
- avoid loading huge datasets into memory

For beginner/intermediate, we’ll use a normal `HttpResponse` writer. Later you’ll
learn streaming exports.

```python
@login_required
@require_GET
def task_export_csv(request, org_slug: str):
    org = get_org_for_user_or_404(user=request.user, org_slug=org_slug)
    membership = get_membership(user=request.user, organization=org)

    if not is_org_admin(membership=membership):
        raise PermissionDenied("Only org admins can export tasks.")

    form = TaskFilterForm(request.GET)
    form.is_valid()

    q = form.cleaned_data.get("q") if form.is_valid() else ""
    status = form.cleaned_data.get("status") if form.is_valid() else ""
    priority = form.cleaned_data.get("priority") if form.is_valid() else ""
    assigned_to = form.cleaned_data.get("assigned_to") if form.is_valid() else ""

    qs = (
        Task.objects.filter(organization=org)
        .select_related("assigned_to", "created_by", "updated_by")
        .order_by("-created_at")
    )

    if q:
        qs = qs.filter(Q(title__icontains=q) | Q(description__icontains=q))
    if status:
        qs = qs.filter(status=status)
    if priority:
        qs = qs.filter(priority=int(priority))
    if assigned_to:
        if assigned_to == "me":
            qs = qs.filter(assigned_to=request.user)
        else:
            qs = qs.filter(assigned_to__username__iexact=assigned_to)

    response = HttpResponse(content_type="text/csv")
    response["Content-Disposition"] = (
        f'attachment; filename="{org.slug}-tasks.csv"'
    )

    writer = csv.writer(response)
    writer.writerow(
        [
            "id",
            "title",
            "status",
            "priority",
            "due_date",
            "assigned_to",
            "created_by",
            "updated_by",
            "created_at",
        ]
    )

    for t in qs:
        writer.writerow(
            [
                t.id,
                t.title,
                t.status,
                t.priority,
                t.due_date.isoformat() if t.due_date else "",
                t.assigned_to.username if t.assigned_to else "",
                t.created_by.username,
                t.updated_by.username,
                t.created_at.isoformat(),
            ]
        )

    return response
```

### Why CSV export is implemented like this
- `Content-Disposition: attachment` triggers file download
- Using the same filters as list prevents “export doesn’t match UI” confusion
- Admin-only avoids data leaks

> Scaling note: for very large exports, you’ll switch to `StreamingHttpResponse` and
> background jobs.

---

## 17.16 Templates (List, Detail, Form, Delete Confirm)

Create template folder:

```text
tasks/templates/tasks/
  list.html
  detail.html
  form.html
  confirm_delete.html
  _pagination.html
```

### 17.16.1 `tasks/list.html`

```django
{% extends "base.html" %}

{% block title %}Tasks — {{ organization.name }}{% endblock %}

{% block content %}
  <h1>Tasks — {{ organization.name }}</h1>

  <p>
    Role: <code>{{ membership.role }}</code>
  </p>

  <p>
    <a href="{% url 'tasks:create' org_slug=organization.slug %}">New task</a>
    {% if can_export %}
      |
      <a href="{% url 'tasks:export_csv' org_slug=organization.slug %}?{{ request.GET.urlencode }}">
        Export CSV
      </a>
    {% endif %}
  </p>

  <form method="get">
    <div class="field">
      <label for="id_q">Search</label>
      <input id="id_q" name="q" value="{{ request.GET.q|default:'' }}" />
    </div>

    <div class="field">
      <label for="id_status">Status</label>
      <select id="id_status" name="status">
        <option value="">Any</option>
        {% for value,label in form.fields.status.choices %}
          {% if value %}
            <option value="{{ value }}" {% if request.GET.status == value %}selected{% endif %}>
              {{ label }}
            </option>
          {% endif %}
        {% endfor %}
      </select>
    </div>

    <div class="field">
      <label for="id_priority">Priority</label>
      <select id="id_priority" name="priority">
        <option value="">Any</option>
        {% for value,label in form.fields.priority.choices %}
          {% if value %}
            <option value="{{ value }}" {% if request.GET.priority == value|stringformat:"s" %}selected{% endif %}>
              {{ label }}
            </option>
          {% endif %}
        {% endfor %}
      </select>
    </div>

    <div class="field">
      <label for="id_assigned_to">Assigned to</label>
      <input
        id="id_assigned_to"
        name="assigned_to"
        value="{{ request.GET.assigned_to|default:'' }}"
        placeholder="me or username"
      />
    </div>

    <button type="submit">Apply</button>
    {% if request.GET %}
      <a href="{% url 'tasks:list' org_slug=organization.slug %}">Clear</a>
    {% endif %}
  </form>

  <hr />

  <ul>
    {% for t in tasks %}
      <li>
        <a href="{% url 'tasks:detail' org_slug=organization.slug task_id=t.id %}">
          {{ t.title }}
        </a>
        —
        <code>{{ t.status }}</code>
        —
        <code>P{{ t.priority }}</code>
        {% if t.assigned_to %}
          —
          assigned to <strong>{{ t.assigned_to.username }}</strong>
        {% endif %}
        {% if t.due_date %}
          —
          due {{ t.due_date }}
        {% endif %}
      </li>
    {% empty %}
      <li>No tasks found.</li>
    {% endfor %}
  </ul>

  {% include "tasks/_pagination.html" with page_obj=page_obj %}
{% endblock %}
```

#### Key pattern: preserve filters in Export link
We used:

```django
?{{ request.GET.urlencode }}
```

So export uses the same filter state as the list.

### 17.16.2 Pagination partial `tasks/_pagination.html`

```django
{% if page_obj.paginator.num_pages > 1 %}
  <nav class="pagination">
    <p>Page {{ page_obj.number }} of {{ page_obj.paginator.num_pages }}</p>

    <div>
      {% if page_obj.has_previous %}
        <a href="?page=1&{{ request.GET.urlencode }}">First</a>
        <a href="?page={{ page_obj.previous_page_number }}&{{ request.GET.urlencode }}">
          Previous
        </a>
      {% endif %}

      {% if page_obj.has_next %}
        <a href="?page={{ page_obj.next_page_number }}&{{ request.GET.urlencode }}">
          Next
        </a>
        <a href="?page={{ page_obj.paginator.num_pages }}&{{ request.GET.urlencode }}">
          Last
        </a>
      {% endif %}
    </div>
  </nav>
{% endif %}
```

> Note: This approach duplicates `page` parameter (you’ll get `page=2&page=3`).
> Advanced fix: write a template tag that rebuilds query string excluding `page`.
> For intermediate, keep it simple; we’ll improve later.

### 17.16.3 `tasks/detail.html`

```django
{% extends "base.html" %}

{% block title %}Task {{ task.id }} — {{ organization.name }}{% endblock %}

{% block content %}
  <h1>{{ task.title }}</h1>

  <p>
    Org: <strong>{{ organization.name }}</strong><br />
    Status: <code>{{ task.status }}</code><br />
    Priority: <code>{{ task.priority }}</code><br />
    Due: {{ task.due_date|default:"(none)" }}<br />
    Assigned to:
    {% if task.assigned_to %}
      {{ task.assigned_to.username }}
    {% else %}
      (unassigned)
    {% endif %}
  </p>

  <p>{{ task.description|linebreaksbr }}</p>

  <hr />

  <p>
    Created by {{ task.created_by.username }} at {{ task.created_at }}<br />
    Updated by {{ task.updated_by.username }} at {{ task.updated_at }}
  </p>

  <p>
    <a href="{% url 'tasks:edit' org_slug=organization.slug task_id=task.id %}">
      Edit
    </a>
    |
    <a href="{% url 'tasks:delete' org_slug=organization.slug task_id=task.id %}">
      Delete
    </a>
    |
    <a href="{% url 'tasks:list' org_slug=organization.slug %}">Back to list</a>
  </p>
{% endblock %}
```

### 17.16.4 `tasks/form.html`

```django
{% extends "base.html" %}

{% block title %}
  {% if mode == "edit" %}Edit Task{% else %}New Task{% endif %}
{% endblock %}

{% block content %}
  <h1>
    {% if mode == "edit" %}Edit Task{% else %}New Task{% endif %}
    — {{ organization.name }}
  </h1>

  <form method="post">
    {% csrf_token %}

    {% if form.non_field_errors %}
      <div class="form-errors">{{ form.non_field_errors }}</div>
    {% endif %}

    <div class="field">
      {{ form.title.label_tag }} {{ form.title }}
      {% for err in form.title.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <div class="field">
      {{ form.description.label_tag }} {{ form.description }}
      {% for err in form.description.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <div class="field">
      {{ form.status.label_tag }} {{ form.status }}
      {% for err in form.status.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <div class="field">
      {{ form.priority.label_tag }} {{ form.priority }}
      {% for err in form.priority.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <div class="field">
      {{ form.due_date.label_tag }} {{ form.due_date }}
      {% for err in form.due_date.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <div class="field">
      {{ form.assigned_to.label_tag }} {{ form.assigned_to }}
      {% for err in form.assigned_to.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <button type="submit">Save</button>
  </form>
{% endblock %}
```

### 17.16.5 `tasks/confirm_delete.html`

```django
{% extends "base.html" %}

{% block title %}Delete Task{% endblock %}

{% block content %}
  <h1>Delete Task</h1>

  <p>Are you sure you want to delete:</p>
  <p><strong>{{ task.title }}</strong></p>

  <form method="post">
    {% csrf_token %}
    <button type="submit">Yes, delete</button>
    <a href="{% url 'tasks:detail' org_slug=organization.slug task_id=task.id %}">
      Cancel
    </a>
  </form>
{% endblock %}
```

---

## 17.17 Seed Data (Quick Manual Setup)

1. Create users in admin (or via `createsuperuser` + admin UI):
   - `alice` (admin)
   - `bob` (member)

2. Create Organization “Acme” with slug `acme`.

3. Create Memberships:
   - alice → acme → ADMIN
   - bob → acme → MEMBER

4. Log in as alice:
   - Create tasks in `/orgs/acme/tasks/new/`

5. Log in as bob:
   - Verify list/visibility
   - Verify edit permission rules on:
     - tasks assigned to bob
     - tasks not assigned and not created by bob

---

## 17.18 Tests (The Most Important Part of Multi‑User Apps)

Create `tasks/tests.py` with scoping + authorization tests.

```python
from django.contrib.auth import get_user_model
from django.test import TestCase
from django.urls import reverse

from orgs.models import Membership, Organization
from tasks.models import Task


class TaskAccessTests(TestCase):
    def setUp(self):
        User = get_user_model()

        self.alice = User.objects.create_user(
            username="alice",
            password="pass12345",
        )
        self.bob = User.objects.create_user(
            username="bob",
            password="pass12345",
        )
        self.mallory = User.objects.create_user(
            username="mallory",
            password="pass12345",
        )

        self.org = Organization.objects.create(name="Acme", slug="acme")

        Membership.objects.create(
            organization=self.org,
            user=self.alice,
            role=Membership.Role.ADMIN,
        )
        Membership.objects.create(
            organization=self.org,
            user=self.bob,
            role=Membership.Role.MEMBER,
        )

        self.task = Task.objects.create(
            organization=self.org,
            title="Admin task",
            description="x",
            status=Task.Status.OPEN,
            priority=Task.Priority.MEDIUM,
            assigned_to=self.bob,
            created_by=self.alice,
            updated_by=self.alice,
        )

    def test_non_member_cannot_access_org(self):
        self.client.login(username="mallory", password="pass12345")
        url = reverse("tasks:list", kwargs={"org_slug": "acme"})
        response = self.client.get(url)
        self.assertEqual(response.status_code, 404)

    def test_member_can_view_list(self):
        self.client.login(username="bob", password="pass12345")
        url = reverse("tasks:list", kwargs={"org_slug": "acme"})
        response = self.client.get(url)
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "Admin task")

    def test_member_can_edit_assigned_task(self):
        self.client.login(username="bob", password="pass12345")
        url = reverse(
            "tasks:edit",
            kwargs={"org_slug": "acme", "task_id": self.task.id},
        )
        response = self.client.get(url)
        self.assertEqual(response.status_code, 200)

    def test_non_assigned_non_creator_member_cannot_edit(self):
        # Create another member and a task not assigned to them.
        User = get_user_model()
        carl = User.objects.create_user(username="carl", password="pass12345")
        Membership.objects.create(
            organization=self.org,
            user=carl,
            role=Membership.Role.MEMBER,
        )

        self.client.login(username="carl", password="pass12345")
        url = reverse(
            "tasks:edit",
            kwargs={"org_slug": "acme", "task_id": self.task.id},
        )
        response = self.client.get(url)
        self.assertEqual(response.status_code, 403)

    def test_admin_can_export_csv(self):
        self.client.login(username="alice", password="pass12345")
        url = reverse("tasks:export_csv", kwargs={"org_slug": "acme"})
        response = self.client.get(url)
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response["Content-Type"], "text/csv")
        self.assertIn("attachment;", response["Content-Disposition"])

    def test_member_cannot_export_csv(self):
        self.client.login(username="bob", password="pass12345")
        url = reverse("tasks:export_csv", kwargs={"org_slug": "acme"})
        response = self.client.get(url)
        self.assertEqual(response.status_code, 403)
```

### Why these tests are “industry standard”
They prove the two most expensive classes of bugs are not present:

- **tenant data leaks** (non-member access)
- **authorization bypass** (member edits someone else’s tasks)
- **sensitive export access** (only admins can export)

---

## 17.19 “History basics”: Add a Simple Task Event Log (Lightweight Audit Trail)

Many companies require “who changed what.” Full audit systems can be complex, but a
minimal event log is extremely useful.

### 17.19.1 Add TaskEvent model

In `tasks/models.py`:

```python
from django.db import models
from django.conf import settings

class TaskEvent(models.Model):
    task = models.ForeignKey(
        Task,
        on_delete=models.CASCADE,
        related_name="events",
    )
    actor = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.PROTECT,
    )
    action = models.CharField(max_length=50)
    created_at = models.DateTimeField(auto_now_add=True)

    # Simple JSON-like data (TextField for SQLite compatibility if needed)
    details = models.JSONField(default=dict, blank=True)

    class Meta:
        ordering = ["-created_at"]
        indexes = [models.Index(fields=["task", "-created_at"])]

    def __str__(self) -> str:
        return f"{self.action} by {self.actor} on task {self.task_id}"
```

Run migrations:

```bash
python manage.py makemigrations
python manage.py migrate
```

### 17.19.2 Log events in edit view (status changes)

In `task_edit`, before saving:

```python
old_status = task.status
old_assignee_id = task.assigned_to_id
```

After save, log if changed:

```python
from tasks.models import TaskEvent

new_status = task.status
new_assignee_id = task.assigned_to_id

if old_status != new_status:
    TaskEvent.objects.create(
        task=task,
        actor=request.user,
        action="status_changed",
        details={"from": old_status, "to": new_status},
    )

if old_assignee_id != new_assignee_id:
    TaskEvent.objects.create(
        task=task,
        actor=request.user,
        action="assignee_changed",
        details={"from": old_assignee_id, "to": new_assignee_id},
    )
```

#### Why this is “history basics”
- It’s not a full diff of every field
- But it captures high-value events
- It’s easy to query and display
- It’s cheap to implement and scales well

---

## 17.20 Exercises (Do these before we move on)

1. **Add membership management UI (non-admin)**
   - Build `/orgs/<slug>/members/` list page
   - Admin-only invite/add (you can do it via admin first)
   - Explain why membership is org-scoped and must not be globally editable

2. **Improve pagination querystring correctness**
   - Create a template tag `querystring` that:
     - starts from `request.GET`
     - replaces `page`
     - removes duplicate page keys
   - Use it in `_pagination.html`

3. **Add “My tasks” quick filter**
   - On list page, add a link that sets `assigned_to=me`
   - Ensure it composes with existing filters

4. **Add index-driven ordering**
   - Add a list mode `?sort=due_date`
   - Ensure you order with nulls last (SQLite behavior differs; document limitations)

5. **Add tests for event logging**
   - Change status in POST edit
   - Assert one `TaskEvent` row created with correct details

---

## 17.21 Project 2 Completion Checklist

- [ ] Orgs + memberships exist and are manageable via admin
- [ ] All task queries are scoped by `organization`
- [ ] Edit/delete/export enforce role and object-level rules
- [ ] Filters/search/pagination work and preserve user intent
- [ ] Audit fields are correct on create/edit
- [ ] CSV export matches list filters
- [ ] Tests verify:
  - non-member cannot access org tasks
  - member cannot edit unauthorized tasks
  - admin can export; member cannot
  - (optional) events logged on changes

---

If you’re ready to continue, the next chapter is **Part IV — 18. Testing (Unit, Integration, E2E)** where we’ll take your existing tests and level them up into an industry-grade testing strategy (factories, mocking, coverage, and query-count regression tests).

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='16. blog_site.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../4. Professional_Django/18 testing.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
